## О чём
В данном ноутбуке реализован алгоритм детекции "замыленных" областей на изображение, описанной в [данной](https://fled.github.io/paper/blur.pdf) статье.

Проект является финальным проектом курса NLA

In [1]:
import cv2
import numpy as np


# getting pixel blur degree to map it
def get_blur_degree(image_file, sv_num=10):
    img = cv2.imread(image_file,cv2.IMREAD_GRAYSCALE)
    u, s, v = np.linalg.svd(img)
    top_sv = np.sum(s[0:sv_num])
    total_sv = np.sum(s)
    return top_sv/total_sv


# gettimg blurred image and segment it
def get_blur_map(image_file, win_size=10, sv_num=3):
    img = cv2.imread(image_file, cv2.IMREAD_GRAYSCALE)
    m, n = img.shape[0]+win_size*2, img.shape[1]+win_size*2
    new_img = np.zeros((m, n))
    new_img[win_size:m-win_size, win_size:n-win_size] = img
    new_img[:win_size, :win_size] = img[0][0]
    new_img[:win_size, n-win_size:] = img[0][-1]
    new_img[m-win_size:, :win_size] = img[-1][0]
    new_img[m-win_size:, n-win_size:] = img[-1][-1]
    new_img[:win_size, win_size:n-win_size] = img[0]
    new_img[win_size:m-win_size, :win_size] = img[:, 0, np.newaxis]
    new_img[win_size:m-win_size, n-win_size:] = img[:, -1, np.newaxis]
    new_img[m-win_size:, win_size:n-win_size] = img[-1]
    
    blur_map = np.zeros((img.shape[0], img.shape[1]))
    max_sv = 0
    min_sv = 1
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            block = new_img[i:i+win_size*2, j:j+win_size*2]
            u, s, v = np.linalg.svd(block)
            top_sv = np.sum(s[0:sv_num])
            total_sv = np.sum(s)
            sv_degree = top_sv/total_sv
            if max_sv < sv_degree:
                max_sv = sv_degree
            if min_sv > sv_degree:
                min_sv = sv_degree
            blur_map[i, j] = sv_degree
    blur_map = (blur_map-min_sv)/(max_sv-min_sv)
    return blur_map


# do tests
def test():
    test1 = './test_images/romashka.jpg'
    test2 = './test_images/oranges.jpg'
    test3 = './test_images/car_no_blur.jpg'
    test4 = './test_images/car_half_blur.jpg'
    test5 = './test_images/car_weak_blur.jpg'
    test6 = './test_images/car_strong_blur.jpg'
    tests = [test1, test2, test3, test4, test5, test6]
    for path in tests:
        result = get_blur_map(path)
        result_name = './test_results/segmented_' + path.split('/')[-1]
        cv2.imwrite(result_name, (1-result)*255)

In [2]:
test()